In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/My Drive

In [0]:
!git clone https://github.com/NVlabs/stylegan.git

In [0]:
cd stylegan

In [0]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

    # Print network details.
    Gs.print_layers()

    # Pick latent vector.
    rnd = np.random.RandomState(100)  # seed = 100
    latents0 = rnd.randn(1, Gs.input_shape[1])
    latents1 = rnd.randn(1, Gs.input_shape[1])
    latents2 = rnd.randn(1, Gs.input_shape[1])
    latents3 = rnd.randn(1, Gs.input_shape[1])
    latents4 = rnd.randn(1, Gs.input_shape[1])
    latents5 = rnd.randn(1, Gs.input_shape[1])
    latents6 = rnd.randn(1, Gs.input_shape[1])
    
    num_split = 39  # 2つのベクトルを39分割
    for i in range(40):
        latents = latents6+(latents0-latents6)*i/num_split
        # Generate image.
        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

        # Save image.
        os.makedirs(config.result_dir, exist_ok=True)
        png_filename = os.path.join(config.result_dir, 'photo'+'{0:04d}'.format(i)+'.png')
        PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

if __name__ == "__main__":
    main()

In [0]:
from PIL import Image
import glob
 
files = sorted(glob.glob('results/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('stylegan.gif', save_all=True, 
               append_images=images[1:], 
               duration=200, loop=0)

In [0]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

url_ffhq        = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)

_Gs_cache = dict()

def load_Gs(url):
    if url not in _Gs_cache:
        with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
            _G, _D, Gs = pickle.load(f)
        _Gs_cache[url] = Gs
    return _Gs_cache[url]

# ----------------  Style mixing -------------------

def draw_style_mixing_figure(png, Gs, w, h, src_seeds, dst_seeds, style_ranges):
    print(png)
    src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
    
    d1 = np.random.RandomState(200).randn(Gs.input_shape[1])  # seed = 200 のベクトルを取得
    d2 = np.random.RandomState(888).randn(Gs.input_shape[1])  # seed = 888 のベクトルを取得
    
    dx = (d2 - d1)/3  #　３分割で補間
    steps = np.linspace(0,3,4)  # stepsに[0,1,2,3] を代入
    dst_latents = np.stack((d1+dx*step) for step in steps)  # dst_latents にベクトルを４つスタック
        
    src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
    dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
    src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
    dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)
    
    canvas = PIL.Image.new('RGB', (w * (len(src_seeds) + 1), h * 5), 'white')
    for col, src_image in enumerate(list(src_images)):
        canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
    for row, dst_image in enumerate(list(dst_images)):
        canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))  
        row_dlatents = np.stack([dst_dlatents[row]] * len(src_seeds))
        row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
        for col, image in enumerate(list(row_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h)) 
             
    png_filename = os.path.join(config.result_dir, 'style_mix.png')
    canvas.save(png_filename)

# --------------- main -----------------

def main():
    tflib.init_tf()
    os.makedirs(config.result_dir, exist_ok=True)
    draw_style_mixing_figure(os.path.join(config.result_dir, 'style_mix.png'), load_Gs(url_ffhq), w=1024, h=1024, 
                                                   src_seeds=[11,701,583], dst_seeds=[888,829,1898,1733,1614,845], 
                                                   style_ranges=[range(0,4)]*4)  # style_mixingのレンジ指定

if __name__ == "__main__":
    main()